## Federated Learning

There are many examples all over the Internet regarding the use of PyTorch in Federated Learning architectures.

The idea is to use a centralized federated learning, in which a central server organizes the training process. They are responsible for choosing the nodes and aggregating the parameters.

Functions: Train clients models, aggregate parameters, test global model, set clients.

**Resources:**

https://towardsdatascience.com/preserving-data-privacy-in-deep-learning-part-3-ae2103c40c22

https://www.kaggle.com/code/puru98/federated-learning-pytorch

https://github.com/yonetaniryo/federated_learning_pytorch/blob/master/FL_pytorch.ipynb


### ID = 1

In [7]:
# Imports
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

The following part is based from the preprocessing process done by Pol Valls in his Bachelor's thesis: 

In [8]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_10456/892409204.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10456/892409204.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10456/892409204.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10456/892409204.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory

In [19]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'normal', data['attack_cat'].replace('normal', 1).replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'dos', data['attack_cat'].replace('dos', 1).replace(['normal', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(2, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace(['normal', 'dos', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(3, 'generic', data['attack_cat'].replace('generic', 1).replace(['normal', 'dos', 'reconnaissance', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(4, 'exploits', data['attack_cat'].replace('exploits', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(5, 'worms', data['attack_cat'].replace('worms', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(6, 'fuzzers', data['attack_cat'].replace('fuzzers', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(7, 'analysis', data['attack_cat'].replace('analysis', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'backdoor', 'shellcode'], 0))
    data_labels.insert(8, 'backdoor', data['attack_cat'].replace('backdoor', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'shellcode'], 0))
    data_labels.insert(9, 'shellcode', data['attack_cat'].replace('shellcode', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor'], 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(21, 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(21, 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(21, 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(21, 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(21, 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(21, 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(21, 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(21, 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(13, 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(13, 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(13, 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(13, 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(13, 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels

    return x, y

Once we have performed the required steps previous to feeding the data to the model, we need to define said model. The idea is to use a CNN model for the local nodes and use averaging of gradients. 

In [20]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    return model 

In [21]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
optimizer = keras.optimizers.Adam(learning_rate=5e-4)
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [22]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID1.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [23]:
def aggregate(grad_list):
    avg_grad = [np.mean(grads, axis = 0) for grads in zip(*grad_list)]
    return avg_grad

In [24]:
global_model = build_model((24,1,1))

In [25]:
x_test, y_test = preprocessing(test_basic)
x_test_plus, y_test_plus = preprocessing(test_plus)

In [26]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(10)], target_names=['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [27]:
# For validation purposes of the global model, a dataset with 15% of samples for each node is taken and merged into a val dataset
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

x1_train, x1_val, y1_train, y1_val = train_test_split(x1, y1, test_size=0.15, random_state=42)
x2_train, x2_val, y2_train, y2_val = train_test_split(x2, y2, test_size=0.15, random_state=42)
x3_train, x3_val, y3_train, y3_val = train_test_split(x3, y3, test_size=0.15, random_state=42)

x_val = np.concatenate((x1_val, x2_val, x3_val), axis=0)
y_val = np.concatenate((y1_val, y2_val, y3_val), axis=0)

In [28]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

best_it = 0

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_weights)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    global_model_it = global_model
    global_model_it.optimizer.apply_gradients(zip(avg_grad, global_model_it.trainable_weights)) # apply gradients to global model
    loss, accuracy, f1, precision, recall = evaluation(global_model_it, x_val, y_val) # Evaluate with validation dataset made before

    loss_it.append(loss)
    accuracy_it.append(accuracy)
    f1_it.append(f1)
    precision_it.append(precision)
    recall_it.append(recall)

    if accuracy_it[-1] == max(accuracy_it): # Choose iterations that has maximized accuracy as the global final model 
        global_model = global_model_it
        best_it = i

global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1.hdf5')

Epoch 1/5
54/54 [==============================] - 21s 316ms/step - loss: 0.7116 - accuracy: 0.8622 - val_loss: 7.5796 - val_accuracy: 0.3314
Epoch 2/5
54/54 [==============================] - 20s 378ms/step - loss: 0.0447 - accuracy: 0.9920 - val_loss: 7.8261 - val_accuracy: 0.3316
Epoch 3/5
54/54 [==============================] - 21s 388ms/step - loss: 0.0282 - accuracy: 0.9926 - val_loss: 8.0394 - val_accuracy: 0.3317
Epoch 4/5
54/54 [==============================] - 21s 398ms/step - loss: 0.0261 - accuracy: 0.9924 - val_loss: 8.2020 - val_accuracy: 0.3317
Epoch 5/5
54/54 [==============================] - 21s 383ms/step - loss: 0.0246 - accuracy: 0.9922 - val_loss: 8.2263 - val_accuracy: 0.3319
Epoch 1/5
53/53 [==============================] - 23s 364ms/step - loss: 0.0228 - accuracy: 0.9927 - val_loss: 1.5222 - val_accuracy: 0.8798
Epoch 2/5
53/53 [==============================] - 16s 300ms/step - loss: 0.0214 - accuracy: 0.9927 - val_loss: 1.5312 - val_accuracy: 0.8837
Epoch 

C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

Epoch 1/5
54/54 [==============================] - 12s 204ms/step - loss: 0.5509 - accuracy: 0.8774 - val_loss: 2.3308 - val_accuracy: 0.3361
Epoch 2/5
54/54 [==============================] - 11s 197ms/step - loss: 0.0677 - accuracy: 0.9928 - val_loss: 4.3840 - val_accuracy: 0.3320
Epoch 3/5
54/54 [==============================] - 11s 199ms/step - loss: 0.0268 - accuracy: 0.9928 - val_loss: 4.7255 - val_accuracy: 0.3323
Epoch 4/5
54/54 [==============================] - 11s 198ms/step - loss: 0.0222 - accuracy: 0.9936 - val_loss: 4.9139 - val_accuracy: 0.3325
Epoch 5/5
54/54 [==============================] - 11s 198ms/step - loss: 0.0208 - accuracy: 0.9939 - val_loss: 5.0295 - val_accuracy: 0.3328
Epoch 1/5
53/53 [==============================] - 11s 180ms/step - loss: 0.0204 - accuracy: 0.9937 - val_loss: 0.9093 - val_accuracy: 0.8818
Epoch 2/5
53/53 [==============================] - 10s 189ms/step - loss: 0.0197 - accuracy: 0.9939 - val_loss: 0.9382 - val_accuracy: 0.8815
Epoch 

C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

54/54 [==============================] - 13s 225ms/step - loss: 0.1076 - accuracy: 0.9939 - val_loss: 4.1863 - val_accuracy: 0.3320
Epoch 2/5
54/54 [==============================] - 13s 249ms/step - loss: 0.0229 - accuracy: 0.9932 - val_loss: 5.0768 - val_accuracy: 0.3324
Epoch 3/5
54/54 [==============================] - 12s 229ms/step - loss: 0.0205 - accuracy: 0.9938 - val_loss: 5.4100 - val_accuracy: 0.3329
Epoch 4/5
54/54 [==============================] - 13s 235ms/step - loss: 0.0196 - accuracy: 0.9941 - val_loss: 5.6450 - val_accuracy: 0.3333
Epoch 5/5
54/54 [==============================] - 12s 230ms/step - loss: 0.0189 - accuracy: 0.9944 - val_loss: 5.7947 - val_accuracy: 0.3334
Epoch 1/5
53/53 [==============================] - 11s 194ms/step - loss: 0.0185 - accuracy: 0.9945 - val_loss: 1.0382 - val_accuracy: 0.8840
Epoch 2/5
53/53 [==============================] - 9s 178ms/step - loss: 0.0178 - accuracy: 0.9947 - val_loss: 1.0644 - val_accuracy: 0.8844
Epoch 3/5
53/53 [

...

C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

54/54 [==============================] - 9s 158ms/step - loss: 0.0542 - accuracy: 0.9914 - val_loss: 2.6402 - val_accuracy: 0.3351
Epoch 2/5
54/54 [==============================] - 9s 160ms/step - loss: 0.0565 - accuracy: 0.9922 - val_loss: 2.9089 - val_accuracy: 0.3333
Epoch 3/5
54/54 [==============================] - 8s 157ms/step - loss: 0.0420 - accuracy: 0.9925 - val_loss: 3.2325 - val_accuracy: 0.3319
Epoch 4/5
54/54 [==============================] - 8s 156ms/step - loss: 0.0351 - accuracy: 0.9925 - val_loss: 3.5917 - val_accuracy: 0.3317
Epoch 5/5
54/54 [==============================] - 8s 155ms/step - loss: 0.0310 - accuracy: 0.9926 - val_loss: 3.9816 - val_accuracy: 0.3315
Epoch 1/5
53/53 [==============================] - 9s 159ms/step - loss: 0.0297 - accuracy: 0.9928 - val_loss: 0.7951 - val_accuracy: 0.8784
Epoch 2/5
53/53 [==============================] - 8s 156ms/step - loss: 0.0280 - accuracy: 0.9928 - val_loss: 0.8288 - val_accuracy: 0.8784
Epoch 3/5
53/53 [======

C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

54/54 [==============================] - 9s 157ms/step - loss: 0.0855 - accuracy: 0.9835 - val_loss: 2.0970 - val_accuracy: 0.3469
Epoch 2/5
54/54 [==============================] - 8s 156ms/step - loss: 0.0976 - accuracy: 0.9805 - val_loss: 2.2452 - val_accuracy: 0.3438
Epoch 3/5
54/54 [==============================] - 8s 154ms/step - loss: 0.0816 - accuracy: 0.9812 - val_loss: 2.4035 - val_accuracy: 0.3412
Epoch 4/5
54/54 [==============================] - 8s 154ms/step - loss: 0.0650 - accuracy: 0.9834 - val_loss: 2.6946 - val_accuracy: 0.3373
Epoch 5/5
54/54 [==============================] - 9s 163ms/step - loss: 0.0475 - accuracy: 0.9874 - val_loss: 3.0314 - val_accuracy: 0.3351
Epoch 1/5
53/53 [==============================] - 9s 166ms/step - loss: 0.0371 - accuracy: 0.9917 - val_loss: 0.6633 - val_accuracy: 0.8787
Epoch 2/5
53/53 [==============================] - 9s 180ms/step - loss: 0.0321 - accuracy: 0.9924 - val_loss: 0.6905 - val_accuracy: 0.8787
Epoch 3/5
53/53 [======

C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

In [29]:
loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, x_test, y_test)
loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, x_test_plus, y_test_plus)

print("Loss for Test Basic: ", loss_basic, " Loss for Test Plus: ", loss_plus)
print("Accuracy for Test Basic: ", accuracy_basic, " Accuracy for Test Plus: ", accuracy_plus)
print("F1 for Test Basic: ", f1_basic, " F1 for Test Plus: ", f1_plus)
print("Precision for Test Basic: ", precision_basic, " Precision for Test Plus: ", precision_plus)
print("Recall for Test Basic: ", recall_basic, " Recall for Test Plus: ", recall_plus)


4279/4279 [==============================] - 16s 4ms/step


C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

1721/1721 [==============================] - 6s 4ms/step
Loss for Test Basic:  0.45142072439193726  Loss for Test Plus:  5.259827136993408
Accuracy for Test Basic:  0.9133410453796387  Accuracy for Test Plus:  0.5523505210876465
F1 for Test Basic:  0.8762276407649864  F1 for Test Plus:  0.5136724085839479
Precision for Test Basic:  0.855519176311318  Precision for Test Plus:  0.48005665650755747
Recall for Test Basic:  0.9133410266148587  Recall for Test Plus:  0.5523505049771126


C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\UX430\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

In [30]:
best_it

0

In [31]:
accuracy_it

[0.9152228832244873,
 0.9146870374679565,
 0.9151903986930847,
 0.914313554763794,
 0.9130632281303406,
 0.9124624729156494,
 0.9119428396224976,
 0.912933349609375,
 0.911991536617279,
 0.9122026562690735]

### ID = 3

In [2]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-B-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_10456/52016006.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10456/52016006.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10456/52016006.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [6]:
x, y = preprocessing(training1)

        srcip   sport  dstip  dsport     spkts  dpkts    sbytes    dbytes  \
0           7  284036      0      83  1.098612      2  4.875197  5.093750   
1           2   99891      3   35080  3.663562     40  7.799343  9.875140   
2           7  295976      4     323  4.812184    126  8.965079  9.603395   
3           1  414853     23     377  2.397895      6  6.776507  5.594711   
4           5  336024      1      83  1.098612      2  4.990433  5.187386   
...       ...     ...    ...     ...       ...    ...       ...       ...   
145195      2  271254      5     323  4.812184    126  8.963544  9.619067   
145196      4  342309      7     128  2.564949     18  7.365813  9.227099   
145197     16    4163      0      83  1.098612      0  5.579730  0.000000   
145198      2  168822      8     128  2.564949     18  7.365813  9.227099   
145199      0   31010      2      83  1.098612      2  4.990433  5.187386   

             stime       ltime  ...  proto_tcp  proto_udp  state_ACC  \
0  

In [71]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID3.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [67]:
global_model = build_model((24,1,1))

In [68]:
x_test, y_test = preprocessing(test_basic)
x_test_plus, y_test_plus = preprocessing(test_plus)

In [ ]:
# For validation purposes of the global model, a dataset with 15% of samples for each node is taken and merged into a val dataset
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

x1_train, x1_val, y1_train, y1_val = train_test_split(x1, y1, test_size=0.15, random_state=42)
x2_train, x2_val, y2_train, y2_val = train_test_split(x2, y2, test_size=0.15, random_state=42)
x3_train, x3_val, y3_train, y3_val = train_test_split(x3, y3, test_size=0.15, random_state=42)

x_val = np.concatenate((x1_val, x2_val, x3_val), axis=0)
y_val = np.concatenate((y1_val, y2_val, y3_val), axis=0)

In [72]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

best_it = 0

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_weights)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    global_model.optimizer.apply_gradients(zip(avg_grad, global_model.trainable_weights)) # apply gradients to global model
    loss, accuracy, f1, precision, recall = evaluation(global_model, x_val, y_val) # Evaluate with validation dataset made before
    loss_it.append(loss)
    accuracy_it.append(accuracy)
    f1_it.append(f1)
    precision_it.append(precision)
    recall_it.append(recall)

global_model.save('C:/Users/UX430/Documents/thesis/code/models/id3.hdf5')

Epoch 1/5
57/57 [==============================] - 12s 205ms/step - loss: 0.1553 - accuracy: 0.9823 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
57/57 [==============================] - 11s 201ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
57/57 [==============================] - 12s 205ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
57/57 [==============================] - 11s 198ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
57/57 [==============================] - 13s 223ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 1/5
53/53 [==============================] - 12s 217ms/step - loss: 3.7189 - accuracy: 0.6432 - val_loss: 1.8616 - val_accuracy: 0.8780
Epoch 2/5
53/53 [==============================] - 11s 212ms/step - loss: 0.0725 - accuracy: 0.9928 - val_loss: 

ValueError: in user code:

    File "c:\Users\UX430\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\UX430\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\UX430\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\UX430\anaconda3\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\UX430\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\UX430\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 

In [ ]:
print("Loss for Test Basic: ", loss_it_basic, " Loss for Test Plus: ", loss_it_plus)
print("Accuracy for Test Basic: ", accuracy_it_basic, " Accuracy for Test Plus: ", accuracy_it_plus)
print("F1 for Test Basic: ", f1_it_basic, " F1 for Test Plus: ", f1_it_plus)
print("Precision for Test Basic: ", precision_it_basic, " Precision for Test Plus: ", precision_it_plus)
print("Recall for Test Basic: ", recall_it_basic, " Recall for Test Plus: ", recall_it_plus)


### ID = 5

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x_test, y_test = preprocessing(test_basic)
x_test_plus, y_test_plus = preprocessing(test_plus)

In [ ]:
# Values saved each iteration 
loss_it_basic = []
accuracy_it_basic = []
f1_it_basic = []
precision_it_basic = []
recall_it_basic = []

loss_it_plus = []
accuracy_it_plus = []
f1_it_plus = []
precision_it_plus = []
recall_it_plus =  []

x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_weights)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    global_model.optimizer.apply_gradients(zip(avg_grad, global_model.trainable_weights)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, x_test, y_test)
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, x_test_plus, y_test_plus)
    # Save values of each iteration
    loss_it_basic.append(loss_basic)
    accuracy_it_basic.append(accuracy_basic)
    f1_it_basic.append(f1_basic)
    precision_it_basic.append(precision_basic)
    recall_it_basic.append(recall_basic)

    loss_it_plus.append(loss_plus)
    accuracy_it_plus.append(accuracy_plus)
    f1_it_plus.append(f1_plus)
    precision_it_plus.append(precision_plus)
    recall_it_plus.append(recall_plus)
global_model.save('C:/Users/UX430/Documents/thesis/code/models/id5.hdf5')